In [1]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.7 MB/s eta 0:00:00


In [2]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

In [ ]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

In [ ]:
env.P[0][3] # Transition model

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [ ]:
env.observation_space.n

16

In [ ]:
env.action_space.n

4

In [3]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [ ]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 18)

In [ ]:

policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 22)

In [ ]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 3)

In [ ]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Right)
SFFF
FHFH
FFFH
HFFG



(0.0, 18)

In [ ]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 10)

In [4]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [ ]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 49/1000
Average number of steps: 10.959183673469388


In [ ]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 105/1000
Average number of steps: 15.17142857142857


In [ ]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 778/1000
Average number of steps: 45.20822622107969


In [5]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [ ]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [ ]:
np.all(v_values_1 >= v_values_0)

True

In [ ]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [ ]:
np.all(v_values_2 >= v_values_1)

True

In [ ]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [ ]:
np.all(v_values_3 >= v_values_2)

True

In [6]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)
    policy = np.zeros(env.observation_space.n, dtype=int)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
            policy[state] = best_action  # save the best action in the policy

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values, policy

In [ ]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [ ]:
optimal_v_values

(array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
        0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
        0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
        0.        ]),
 array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0]))

In [7]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [ ]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [ ]:
optimal_policy

In [ ]:
play(env, optimal_policy, True)

In [ ]:
play_multiple_times(env, optimal_policy, 1000)

In [8]:
import numpy as np
import gymnasium as gym

def policy_iteration(env, gamma=0.9, max_iters=500):
    # initialize a random policy or all zeros policy
    policy = np.zeros(env.observation_space.n, dtype=int)
    for i in range(max_iters):
        # evaluate the current policy
        v_values = policy_evaluation(env, policy, max_iters, gamma)

        # extract a new policy based on the updated value function
        new_policy = policy_extraction(env, v_values, gamma)

        # check if the new policy is the same as the old policy
        if np.all(new_policy == policy):
            break

        policy = new_policy

    return policy, v_values

# # create the environment
# env = gym.make('FrozenLake-v1', render_mode="ansi")

# # run the policy iteration
# optimal_policy, optimal_v_values = policy_iteration(env)
# print("Optimal Policy:", optimal_policy)
# print("Optimal Value Function:", optimal_v_values)

In [ ]:
environments = ['FrozenLake-v1', 'FrozenLake8x8-v1', 'Taxi-v3']


In [16]:
import time
i='FrozenLake-v1'
s_policy_iteration = 0
t_i = time.perf_counter()
env = gym.make(i, render_mode="ansi")
optimal_policy, optimal_v_values = policy_iteration(env)
print(play(env, optimal_policy, True))
t_f = time.perf_counter()
s_policy_iteration += t_f - t_i
print("Thời gian chạy:",t_f - t_i,"s")


  (Down)
SFFF
FHFH
FFFH
HFFG

(1.0, 50)
Thời gian chạy: 25.200034587000005 s


In [10]:
environments = ['FrozenLake-v1', 'FrozenLake8x8-v1', 'Taxi-v3']
i='FrozenLake-v1'
s_value_iteration = 0
t_i = time.perf_counter()
env = gym.make(i, render_mode="ansi")
optimal_value, policy = value_iteration(env, max_iters=500, gamma=0.9)
print(play(env, policy, True))
t_f = time.perf_counter()
s_value_iteration += t_f - t_i
print(f"Thời gian chạy:{t_f - t_i}s")


  (Down)
SFFF
FHFH
FFFH
HFFG

(1.0, 42)
Thời gian chạy:21.203350549999982s


In [17]:
i='FrozenLake8x8-v1'
s_policy_iteration = 0
t_i = time.perf_counter()
env = gym.make(i, render_mode="ansi")
optimal_policy, optimal_v_values = policy_iteration(env)
print(play(env, optimal_policy, True))
t_f = time.perf_counter()
s_policy_iteration += t_f - t_i
print("Thời gian chạy:",t_f - t_i,"s")

  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

(0.0, 22)
Thời gian chạy: 12.517198558000018 s


In [20]:
i='FrozenLake8x8-v1'
s_value_iteration = 0
t_i = time.perf_counter()
env = gym.make(i, render_mode="ansi")
optimal_value, policy = value_iteration(env, max_iters=500, gamma=0.9)
print(play(env, policy, True))
t_f = time.perf_counter()
s_value_iteration += t_f - t_i
print(f"Thời gian chạy:{t_f - t_i}s")

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

(1.0, 62)
Thời gian chạy:31.64873072799992s


In [13]:
i='Taxi-v3'
s_policy_iteration = 0
t_i = time.perf_counter()
env = gym.make(i, render_mode="ansi")
optimal_policy, optimal_v_values = policy_iteration(env)
print(play(env, optimal_policy, True))
t_f = time.perf_counter()
s_policy_iteration += t_f - t_i
print("Thời gian chạy:",t_f - t_i,"s")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

(7, 14)
Thời gian chạy: 21.855840166999997 s


In [21]:
i='Taxi-v3'
s_value_iteration = 0
t_i = time.perf_counter()
env = gym.make(i, render_mode="ansi")
optimal_value, policy = value_iteration(env, max_iters=500, gamma=0.9)
print(play(env, policy, True))
t_f = time.perf_counter()
s_value_iteration += t_f - t_i
print(f"Thời gian chạy:{t_f - t_i}s")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

(5, 16)
Thời gian chạy:13.550028194999982s


In [ ]:
# Các em có nhận xét gì so sánh Policy Iteration và Value Iteration?
# Trong các toy games ở đây thì thuật toán nào cho ra kết quả nhanh hơn (trung bình thời gian chạy)?

# Viết ngắn gọn nhận xét và kết quả (dạng comment) vào file ipynb Jupyter notebook.

In [ ]:
# nhận xét gì so sánh Policy Iteration và Value Iteration?
# Policy Iteration:
# - Gồm 2 bước chính : policy evaluation and policy improvement.
# - Thay thế giữa việc đánh giá 1 policy nhất định và cải thiện nó bằng cách làm cho nó trở nên tham lam đối với hàm giá trị của nó.
# - Đạt được sự hội tụ khi policy không thay đổi sau một bước extraction,
# thường yêu cầu ít lần lặp hơn nhưng mỗi lần lặp lại liên quan đến
# việc giải một hệ phương trình (sao lưu toàn bộ chính sách).
# - Có thể chậm hơn đối với các môi trường có không gian trạng thái lớn hoặc khi bước đánh giá yêu cầu nhiều lần lặp để hội tụ.
# Value Iteration:
# - Đơn giản hóa quy trình bằng cách kết hợp policy evaluation và extraction thành một bước duy nhất.
# - Tính toán trực tiếp hàm giá trị tối ưu bằng cách xem xét tất cả các hành động ở mỗi trạng thái rồi chọn giá trị tối đa.
# - Thường hội tụ nhanh hơn policy iteration về số lần lặp vì nó liên tục cập nhật các ước tính giá trị về giá trị tối ưu.
# - Mỗi lần lặp lại có khả năng nhanh hơn vì nó tránh được khả năng hội tụ chậm trong việc policy evalution.

In [ ]:
# Trong các toy games ở đây thì thuật toán nào cho ra kết quả nhanh hơn (trung bình thời gian chạy)?
"""
Kết quả của các toy games khi lần lượt thực hiện các thuật toán Policy Iteration và Value Iteration.
'FrozenLake-v1': 25.200034587000005s(Số điểm đạt được:1.0, số bước:50)(Với Policy Iteration) và 21.203350549999982s(Số điểm đạt được:1.0, số bước:42)(với Value Iteration).
'FrozenLake8x8-v1':12.517198558000018s(Số điểm đạt được:0.0, số bước:22)(Với Policy Iteration) và 31.64873072799992s(Số điểm đạt được:1.0, số bước:62)(với Value Iteration).
'Taxi-v3':21.855840166999997s(Số điểm đạt được:7, số bước:14)(Với Policy Iteration) và 13.550028194999982s(Số điểm đạt được:5, số bước:16)(với Value Iteration).
Nhận xét:
-Thuật toán Value Iteration cho ra kết quả nhanh hơn(bằng việc policy evaluation và extraction thành một bước duy nhất, nhờ đó giảm thời gian chạy) ở hầu hết các toy games('FrozenLake-v1' và 'Taxi-v3').
-Thuật toán Value Iteration cũng không có kết quả thấp hơn Policy Iteration ở hầu hết các toy games('FrozenLake-v1' và 'FrozenLake8x8-v1').
"""